In [226]:
#data
import numpy as np
import pandas as pd
import sqlite3

#text
from bs4 import BeautifulSoup
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import spacy
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#aux
from IPython.display import clear_output
import os
import pickle

# essas duas linhas são usadas para definir como os números de ponto flutuante 
# serão formatados ao serem exibidos em numpy e pandas, garantindo que eles sejam 
# exibidos com uma precisão específica e sem notação científica.'''
np.set_printoptions(suppress=True, formatter={'float_kind':'{:16.3f}'.format}, linewidth=130)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## carregar os dados

In [78]:
file = os.path.join('data', 'df_itens_geral_validos.csv')
df_itens = pd.read_csv(file, sep = ';', encoding = 'UTF-8')

In [79]:
df_itens.shape #1922 itens

(1922, 28)

In [80]:
df_itens.head()

,co_posicao,sg_area,co_item,tx_gabarito,co_habilidade,in_item_aban,tx_motivo_aban,nu_param_a,nu_param_b,nu_param_c,...,texto_questao,texto_base,alternativa_a,alternativa_b,alternativa_c,alternativa_d,alternativa_e,tp_lingua,in_item_adaptado,tp_versao_digital
0,1,CH,71933,A,6.00000,0,NaN,2.23777,1.04258,0.29804,...,O gráfico representa a relação entre o tamanho...,O gráfico representa a relação entre o tamanh...,A concentraçao de terras nas mãos de poucos.,A existência de poucas terras agricultáveis.,O domínio territorial dos minifúndios.,A primazia da agricultura familiar.,A debilidade dos plantations modernos.,NaN,NaN,NaN
1,2,CH,71990,B,19.00000,0,NaN,1.35599,1.23358,0.23895,...,"Antes, eram apenas as grandes cidades que se a...","Antes, eram apenas as grandes cidades que se a...",a redução do processo de concentração de terras.,o aumento do aproveitamento de solos menos fé...,a ampliação do isolamento do espaço rural.,a estagnação da fronteira agrícola do país.,a diminuição do nível de emprego formal.,NaN,NaN,NaN
2,3,CH,73431,A,17.00000,0,NaN,1.58426,1.26006,0.19016,...,A maioria das pessoas daqui era do campo. Vila...,A maioria das pessoas daqui era do campo. Vila...,dos impactos sociais da modernização da agric...,da recomposição dos salários do trabalhador r...,da exigência de qualificação do trabalhador r...,da diminuição da importância da agricultura.,dos processos de desvalorização de áreas rurais.,NaN,NaN,NaN
3,4,CH,58249,B,26.00000,0,NaN,1.87124,-0.40493,0.17401,...,Os lixões são o pior tipo de disposição final ...,Os lixões são o pior tipo de disposição final ...,"causam problemas respiratórios, devido ao mau...",são locais propícios à proliferação de vetore...,"provocam o fenômeno da chuva ácida, devido ao...",são instalados próximos ao centro das cidades...,são responsáveis pelo desaparecimento das nas...,NaN,NaN,NaN
4,5,CH,73854,D,27.00000,0,NaN,0.40830,1.46209,0.02447,...,O esquema representa um processo de erosão em ...,O esquema representa um processo de erosão em...,Plantio direto.,Associação de culturas.,Implantação de curvas de nível.,"Aração do solo, do topo ao vale.",Terraceamento na propriedade,NaN,NaN,NaN


In [81]:
data = df_itens

## pré-processamento dos textos dos itens

## carregar o modelo de word embbeding pré-treinado

Um **modelo de word embedding pré-treinado** é um modelo de aprendizado de máquina que foi previamente treinado com uma grande quantidade de dados textuais. A ideia é que, através do processo de treinamento, o modelo aprenda a representar as palavras (ou tokens) de um texto como vetores de números, de modo que as palavras similares tenham vetores similares. Estes modelos podem ser usados para diversas tarefas de processamento de linguagem natural, incluindo classificação de texto, análise de sentimentos, entre outros. Alguns modelos pré-treinados são disponibilizados pelas bibliotecas de aprendizado de máquina e podem ser facilmente utilizados sem a necessidade de treinar novamente o modelo.

O **"cc.pt.300"** é um modelo de word embedding pré-treinado para a língua portuguesa desenvolvido pela FastText, uma biblioteca para trabalhar com processamento de texto em larga escala.

Características deste modelo incluem:

- É treinado em milhões de sentenças e documentos na língua portuguesa;
- Utiliza o algoritmo de subpalavras (subword), o que permite a representação de palavras semelhantes mesmo que não estejam presentes no treinamento;
- A dimensão do word embedding é 300, o que significa que cada palavra é representada por um vetor de 300 números;
- Fornece uma representação de palavras com similaridade semântica e semântica próxima.

Este modelo é amplamente utilizado para tarefas de processamento de texto, como análise de sentimentos, classificação de textos, clusterização de documentos, etc.

In [ ]:
#carregar o modelo
file = os.path.join('data', 'cc.pt.300.vec')
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)

# armazenar todas as palavras e embbedings do modelo pré-treinado em um banco de dados sqlite
file = os.path.join('data', 'embeddings.db')
conn = sqlite3.connect(file)

cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS embeddings (
    word TEXT PRIMARY KEY,
    embedding BLOB
)
""")

#iterar sobre as words armazenando essas words
for word in fasttext_model.index_to_key:
    cursor.execute("""
    INSERT INTO embeddings (word, embedding)
    VALUES (?,?)
    """, (word, fasttext_model[word].tobytes()))

conn.commit()
conn.close()

In [41]:
# função para buscar o word embbeding de uma paravra em um banco de dados sqlite 
# sem precisar carregar na memória modelo de word embedding pré-treinado
def get_sqlite_we(word):
    # Conectar ao banco de dados
    file = os.path.join('data', 'embeddings.db')
    conn = sqlite3.connect(file)
    cursor = conn.cursor()

    # Executar uma consulta que seleciona a coluna com o BLOB
    cursor.execute("SELECT embedding FROM embeddings WHERE word = '"+word+"'")

    # Obter o resultado da consulta
    result = cursor.fetchone()
    
    embedding = np.array([])
    if result != None:
        # Converter o BLOB em array NumPy
        embedding = np.frombuffer(result[0], dtype=np.float32)

    conn.commit()
    conn.close()
    
    return embedding

In [174]:
# realizar o pré-processamento de um texto
def pre_processa(row, row_index):
    # carregar o modelo de linguagem em português
    #text = row.texto_questao
    text = row
    nlp = spacy.load("pt_core_news_md")
    stop_words = set(stopwords.words("portuguese"))
    text = text.replace('em:', 'em: ')
    # remover urls
    text = re.sub(r"/\b(?:(?:https?|ftp):\/\/|www\.)[-a-z0-9+&@#\/%?=~_|!:,.;]*[-a-z0-9+&@#\/%=~_|]/i", ' ', text)
    text = re.sub(r'^(?!http)\S+', '', text)
    text = text.replace('.', ' . ')
    #transformar em minúscula
    text = text.lower()
    #tokenizar
    text_tokens = word_tokenize(text)
    #remover stop words
    text_tokens = [word.lower() for word in text_tokens if word.lower() not in stop_words]
    #lematizar
    text_tokens = [nlp(word)[0].lemma_ for word in text_tokens]
    #remover pontuações
    text_tokens = [word for word in text_tokens if word not in string.punctuation]
    #verificar se a palavra pertence ao modelo de word embedding pré-treinado
    words = []
    wes = []
    for s in text_tokens: 
        if ('alva') in s:
            we = get_sqlite_we('alva')
        elif ('água') in s:
            we = get_sqlite_we('água')
        elif ('enlaça') in s:
            we = get_sqlite_we('enlaça')
        elif ('roa') in s:
            we = get_sqlite_we('roa')
        else:
            we = get_sqlite_we(s)
        if len(we) > 0:
            words.append(s)
            wes.append(list(we))
    text = ' '.join(words)
    clear_output(wait=True)
    print(row_index, text)
    return text, wes

A **medida tf-idf** (term frequency-inverse document frequency) é usada para avaliar a importância relativa de uma palavra em um documento dentro de uma coleção de documentos. A partir da contagem das ocorrências de cada palavra em um documento, a medida tf calcula a frequência relativa de cada palavra em relação ao número total de palavras no documento, enquanto a medida idf avalia a raridade da palavra na coleção de documentos. Assim, a medida tf-idf calcula o produto da medida tf pela medida idf, fornecendo um valor numérico que indica a relevância de uma palavra em um documento específico em relação aos outros documentos da coleção.

Com base na medida tf-idf, pode-se inferir sobre a importância relativa de uma palavra em um documento específico em comparação com outros documentos da coleção. Isso pode ser útil em tarefas como recuperação de informações, classificação de documentos e análise de sentimentos, onde a importância das palavras em relação ao contexto pode fornecer insights sobre o conteúdo dos documentos. Além disso, a medida tf-idf pode ser usada para identificar palavras-chave e tópicos relevantes em um documento, bem como para a remoção de palavras comuns e irrelevantes (stopwords) na análise de texto.

A faixa de valores da medida tf-idf pode variar de 0 a um valor positivo, dependendo da frequência de termos nos documentos e na coleção de documentos. O valor 0 indica que o termo não aparece no documento ou que o termo aparece em todos os documentos da coleção. Quanto maior o valor tf-idf, mais relevante é o termo para o documento.

In [222]:
#obtendo os vetores de TF-IDF
def add_tf_idf(dataset, col, filtro = None):
    texts = ''
    indexs = []
    print(filtro)
    if (filtro != None):
        texts = dataset[dataset[filtro[0]] == filtro[1]][col].values
        indexs = dataset[dataset[filtro[0]] == filtro[1]].index.values
        dataset.loc[indexs, 'tf_idf'] = ''
    else:
        texts = dataset[col].values
        indexs = dataset.index.values
        dataset.loc[indexs, 'tf_idf'] = ''
        
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(texts)

    # Converter a matriz esparsa em uma matriz densa (numpy)
    dense_matrix = vectors.todense()

    # Criar o DataFrame a partir da matriz densa
    df = pd.DataFrame(dense_matrix)
    print(df.shape)
    for i, index in enumerate(indexs):
        #print(df.iloc[i, :].values[np.nonzero(df.iloc[i, :].values)])
        l = list(df.iloc[i, :].values[np.nonzero(df.iloc[i, :].values)])
        dataset.at[index, 'tf_idf'] = l

    return dataset

In [124]:
data['texto_questao_tratado'] = np.nan
data['wes'] = ''

In [140]:
data.head()

,co_posicao,sg_area,co_item,tx_gabarito,co_habilidade,in_item_aban,tx_motivo_aban,nu_param_a,nu_param_b,nu_param_c,...,alternativa_a,alternativa_b,alternativa_c,alternativa_d,alternativa_e,tp_lingua,in_item_adaptado,tp_versao_digital,texto_questao_tratado,wes
0,1,CH,71933,A,6.00000,0,NaN,2.23777,1.04258,0.29804,...,A concentraçao de terras nas mãos de poucos.,A existência de poucas terras agricultáveis.,O domínio territorial dos minifúndios.,A primazia da agricultura familiar.,A debilidade dos plantations modernos.,NaN,NaN,NaN,gráfico representar relação tamanho totalidade...,"[[0.0087, -0.015, 0.073, 0.0244, -0.0139, -0.0..."
1,2,CH,71990,B,19.00000,0,NaN,1.35599,1.23358,0.23895,...,a redução do processo de concentração de terras.,o aumento do aproveitamento de solos menos fé...,a ampliação do isolamento do espaço rural.,a estagnação da fronteira agrícola do país.,a diminuição do nível de emprego formal.,NaN,NaN,NaN,apenas grande cidade apresentar império técnic...,"[[-0.0457, -0.009, 0.0753, -0.0198, -0.0479, -..."
2,3,CH,73431,A,17.00000,0,NaN,1.58426,1.26006,0.19016,...,dos impactos sociais da modernização da agric...,da recomposição dos salários do trabalhador r...,da exigência de qualificação do trabalhador r...,da diminuição da importância da agricultura.,dos processos de desvalorização de áreas rurais.,NaN,NaN,NaN,maioria pessoa campo Vila Maria hoje exportado...,"[[-0.0246, -0.0379, 0.1021, 0.0378, -0.0864, -..."
3,4,CH,58249,B,26.00000,0,NaN,1.87124,-0.40493,0.17401,...,"causam problemas respiratórios, devido ao mau...",são locais propícios à proliferação de vetore...,"provocam o fenômeno da chuva ácida, devido ao...",são instalados próximos ao centro das cidades...,são responsáveis pelo desaparecimento das nas...,NaN,NaN,NaN,lixão mau tipo disposição final resíduo sólido...,"[[-0.0016, -0.0689, 0.0547, -0.0424, 0.0693, -..."
4,5,CH,73854,D,27.00000,0,NaN,0.40830,1.46209,0.02447,...,Plantio direto.,Associação de culturas.,Implantação de curvas de nível.,"Aração do solo, do topo ao vale.",Terraceamento na propriedade,NaN,NaN,NaN,esquema representar processo erosão encosta pr...,"[[-0.0128, -0.0053, 0.029, -0.052, -0.0304, -0..."


In [175]:
# tratar todos os textos e obter o word embbeddings de todos os textos
for i in [388, 741, 1362]:
    texto_tratado, wes = pre_processa(data.at[i, 'texto_questao'], i+1)
    data.at[i, 'texto_questao_tratado'] = texto_tratado
    data.at[i, 'wes'] = wes

1363 laço sabes criança louco amor prendi afeto formoso pepita onde templo espaço névoa ria prendi-me laço fita selva sombrio madeixa negro cabelo moça bonito fingir serpente qu'enlaça folhagem formoso enroscava-se laço fita pois bem dia sombra valer cova formoso pepita menos arrancar louro fronte dá-me c'roa laço fita Alves c espumas flutuante disponível www gov br acesso 8 ago fragmento exemplo lírico temática amoroso castro Alves poema constrói imagem cara fragmento lirismo romântico expressar naa representação infantilizar figura feminino b criatividade inspirar elemento natureza c opção morte solução frustração d ansiedade atitude indiferença mulher fixação signo fusão simbólico amar


In [202]:
data['sg_area'].unique()

array(['CH', 'CN', 'LC', 'MT'], dtype=object)

In [223]:
# tf idf por área
add_tf_idf(data, 'texto_questao_tratado', ['sg_area', 'LC'])
add_tf_idf(data, 'texto_questao_tratado', ['sg_area', 'MT'])
add_tf_idf(data, 'texto_questao_tratado', ['sg_area', 'CN'])
add_tf_idf(data, 'texto_questao_tratado', ['sg_area', 'CH'])

['sg_area', 'LC']
(442, 9667)
['sg_area', 'MT']
(509, 4907)
['sg_area', 'CN']
(484, 6236)
['sg_area', 'CH']
(487, 7975)


,co_posicao,sg_area,co_item,tx_gabarito,co_habilidade,in_item_aban,tx_motivo_aban,nu_param_a,nu_param_b,nu_param_c,...,alternativa_b,alternativa_c,alternativa_d,alternativa_e,tp_lingua,in_item_adaptado,tp_versao_digital,texto_questao_tratado,wes,tf_idf
0,1,CH,71933,A,6.00000,0,NaN,2.23777,1.04258,0.29804,...,A existência de poucas terras agricultáveis.,O domínio territorial dos minifúndios.,A primazia da agricultura familiar.,A debilidade dos plantations modernos.,NaN,NaN,NaN,gráfico representar relação tamanho totalidade...,"[[0.0087, -0.015, 0.073, 0.0244, -0.0139, -0.0...","[0.15448001662671257, 0.19314334362072388, 0.1..."
1,2,CH,71990,B,19.00000,0,NaN,1.35599,1.23358,0.23895,...,o aumento do aproveitamento de solos menos fé...,a ampliação do isolamento do espaço rural.,a estagnação da fronteira agrícola do país.,a diminuição do nível de emprego formal.,NaN,NaN,NaN,apenas grande cidade apresentar império técnic...,"[[-0.0457, -0.009, 0.0753, -0.0198, -0.0479, -...","[0.14463912531808382, 0.1882090195082598, 0.11..."
2,3,CH,73431,A,17.00000,0,NaN,1.58426,1.26006,0.19016,...,da recomposição dos salários do trabalhador r...,da exigência de qualificação do trabalhador r...,da diminuição da importância da agricultura.,dos processos de desvalorização de áreas rurais.,NaN,NaN,NaN,maioria pessoa campo Vila Maria hoje exportado...,"[[-0.0246, -0.0379, 0.1021, 0.0378, -0.0864, -...","[0.08937166525923214, 0.12822456358498585, 0.1..."
3,4,CH,58249,B,26.00000,0,NaN,1.87124,-0.40493,0.17401,...,são locais propícios à proliferação de vetore...,"provocam o fenômeno da chuva ácida, devido ao...",são instalados próximos ao centro das cidades...,são responsáveis pelo desaparecimento das nas...,NaN,NaN,NaN,lixão mau tipo disposição final resíduo sólido...,"[[-0.0016, -0.0689, 0.0547, -0.0424, 0.0693, -...","[0.07309805460729854, 0.07309805460729854, 0.0..."
4,5,CH,73854,D,27.00000,0,NaN,0.40830,1.46209,0.02447,...,Associação de culturas.,Implantação de curvas de nível.,"Aração do solo, do topo ao vale.",Terraceamento na propriedade,NaN,NaN,NaN,esquema representar processo erosão encosta pr...,"[[-0.0128, -0.0053, 0.029, -0.052, -0.0304, -0...","[0.238915125308662, 0.2103826054875806, 0.2674..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917,115,CN,111647,B,23.00000,0,NaN,2.72669,1.99105,0.20284,...,"alumínio, cobre-aço e ferro.","ompara-se uma panela de ferro, uma de alumínio...","alumínio, ferro e cobre-aço.","ferro, alumínio e cobre-aço.",NaN,NaN,NaN,montagem cozinha restaurante escolha material ...,"[[-0.0141, 0.0021, 0.0354, -0.0111, -0.0444, -...","[0.05619102715705249, 0.06916741900157027, 0.0..."
1918,129,CN,87718,B,16.00000,0,NaN,3.29305,1.87858,0.17777,...,reserva para o embrião.,revestimento para a semente.,proteção para o megagametófito.,ssa segunda fertilização foi importante evolut...,NaN,NaN,NaN,angiosperma além fertilização oosfera existir ...,"[[0.0061, -0.0594, -0.0064, 0.0171, 0.0129, -0...","[0.1528893527738725, 0.2600060925862451, 0.209..."
1919,92,CN,63095,D,1.00000,0,NaN,3.01758,1.65693,0.15293,...,f < f e v = v. 1 2 1 2,"onsidere f e v, respectivamente, como a frequê...",f > f e v = v. 1 2 1 2,f > f e v > v. 1 2 1 2,NaN,NaN,NaN,sino vento compor várias barra metálico materi...,"[[0.1122, -0.1142, 0.174, -0.0254, 0.0135, -0....","[0.11929146186934009, 0.19125392140927147, 0.5..."
1920,104,CN,117795,B,10.00000,0,NaN,3.24877,1.21839,0.09433,...,golfinhos.,"ommission, Conference Paper , jun. 2016 (adapt...",peixes herbívoros.,algas macroscópicas.,NaN,NaN,NaN,rompimento barragem rejeito mineração municípi...,"[[0.0164, -0.0423, 0.0183, 0.0021, -0.0077, -0...","[0.17188810429433815, 0.14762382523360137, 0.1..."


## treinar um word embbeding com o corpus de questões

Para treinar um modelo de Word Embedding próprio com uma coleção de documentos de texto, podemos utilizar a biblioteca Gensim em Python.

A função build_vocab é usada para criar o vocabulário do modelo, que é a lista de todas as palavras únicas presentes no corpus com frequência superior a min_count. Finalmente, o modelo é treinado usando a função train. O argumento total_examples é usado para especificar o número total de documentos no corpus e o argumento epochs é usado para especificar o número de épocas de treinamento.

O treinamento do modelo pode levar algum tempo, dependendo do tamanho do corpus e dos parâmetros do modelo. O callback MonitorCallback é usado para imprimir uma mensagem após cada época de treinamento para monitorar o progresso.

Depois de treinar o modelo, é possível acessar os vetores de Word Embedding de cada palavra no vocabulário usando a propriedade model.wv. Por exemplo, model.wv['palavra'] retorna o vetor de Word Embedding correspondente à palavra 'palavra'.

In [240]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

# Classe de callback para monitorar o treinamento do modelo
class MonitorCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
    
    def on_epoch_end(self, model):
        print(f'Epoch {self.epoch} concluída')
        self.epoch += 1

corpus = [doc.lower().split() for doc in data.texto_questao_tratado.values]

# Definir os parâmetros do modelo de Word Embedding
size = 100  # tamanho do vetor de embedding
window = 5  # tamanho da janela de contexto
min_count = 1  # frequência mínima das palavras no corpus para serem consideradas no vocabulário
sg = 1  # utilizar o algoritmo Skip-gram

# Inicializar o modelo de Word Embedding
model = Word2Vec(vector_size=size, window=window, min_count=min_count, sg=sg)

# Treinar o modelo de Word Embedding
monitor_callback = MonitorCallback()
model.build_vocab(corpus)
model.train(corpus, total_examples=len(corpus), epochs=10, callbacks=[monitor_callback])

Epoch 0 concluída
Epoch 1 concluída
Epoch 2 concluída
Epoch 3 concluída
Epoch 4 concluída
Epoch 5 concluída
Epoch 6 concluída
Epoch 7 concluída
Epoch 8 concluída
Epoch 9 concluída


(1621991, 1669540)

In [260]:
file = os.path.join('data', 'we_model_enem.bin')
model.save(file)

In [261]:
# carrega o modelo treinado
model = Word2Vec.load(file)

# recupera as palavras do vocabulário
vocab_words = model.wv.index_to_key

# grava as palavras em um arquivo de texto
file = os.path.join('data', 'vocab.txt')
with open(file, "w", encoding="utf-8") as f:
    f.write("\n".join(vocab_words))

In [241]:
model.wv['concentraçao']

array([          -0.081,            0.084,            0.016,            0.116,           -0.024,           -0.135,
                  0.048,            0.249,            0.040,           -0.039,           -0.121,           -0.122,
                 -0.042,            0.038,           -0.017,           -0.064,            0.057,           -0.103,
                  0.037,           -0.161,            0.021,           -0.009,            0.048,           -0.025,
                  0.005,           -0.013,           -0.136,           -0.119,           -0.033,            0.052,
                  0.200,            0.076,           -0.008,           -0.012,           -0.007,            0.088,
                  0.044,           -0.130,           -0.072,           -0.138,            0.044,           -0.138,
                  0.038,           -0.102,            0.080,           -0.025,           -0.054,            0.061,
                  0.017,            0.067,            0.154,           -0.120,  

In [245]:
data['wes_own'] = ''
for i, questao in enumerate(corpus):
    print(i)
    wes_own = []
    print(questao)
    for s in questao: 
        we = model.wv[s]
        if len(we) > 0:
            wes_own.append(list(we))
    data.at[i, 'wes_own'] = wes_own
    clear_output(wait=True)

1921
['escola', 'iniciar', 'processo', 'educativo', 'implantação', 'coleta', 'seletivo', 'material', 'reciclável', 'atingir', 'objetivo', 'instituição', 'sensibilizar', 'comunidade', 'escol', 'ar', 'desenvolver', 'atividade', 'sala', 'maneira', 'contínuo', '2', 'capacitar', 'pessoal', 'responsá', 'vel', 'limpeza', 'escola', 'quanto', 'adotar', 'coleta', 'seletivo', 'e3', 'distribuir', 'coletor', 'material', 'reciclável', 'específico', 'sala', 'pátio', 'acondicionamento', 'resíduo', 'completar', 'ação', 'proposta', 'ambiente', 'escolar', 'faltar', 'arealizar', 'campanha', 'educativo', 'sensibilização', 'bairro', 'vizinho', 'coleta', 'seletivo', 'parceria', 'prefeitura', 'cooperativa', 'catador', 'material', 'reciclável', 'destinação', 'apropriado', 'visita', 's', 'lixão', 'aterro', 'local', 'identificar', 'aspecto', 'disposição', 'final', 'lixo', 'rádio', 'local', 'jorna', 'l', 'impresso', 'rede', 'social', 'escola', 'coleta', 'seletivo', 'ecolocar', 'tes', 'coletor', 'lixo', 'recicláve

In [246]:
data.head()

,co_posicao,sg_area,co_item,tx_gabarito,co_habilidade,in_item_aban,tx_motivo_aban,nu_param_a,nu_param_b,nu_param_c,...,alternativa_c,alternativa_d,alternativa_e,tp_lingua,in_item_adaptado,tp_versao_digital,texto_questao_tratado,wes,tf_idf,wes_own
0,1,CH,71933,A,6.00000,0,NaN,2.23777,1.04258,0.29804,...,O domínio territorial dos minifúndios.,A primazia da agricultura familiar.,A debilidade dos plantations modernos.,NaN,NaN,NaN,gráfico representar relação tamanho totalidade...,"[[0.0087, -0.015, 0.073, 0.0244, -0.0139, -0.0...","[0.15448001662671257, 0.19314334362072388, 0.1...","[[-0.2735036, -0.19421376, 0.26711008, 0.26224..."
1,2,CH,71990,B,19.00000,0,NaN,1.35599,1.23358,0.23895,...,a ampliação do isolamento do espaço rural.,a estagnação da fronteira agrícola do país.,a diminuição do nível de emprego formal.,NaN,NaN,NaN,apenas grande cidade apresentar império técnic...,"[[-0.0457, -0.009, 0.0753, -0.0198, -0.0479, -...","[0.14463912531808382, 0.1882090195082598, 0.11...","[[0.33006182, 0.08265074, 0.0863686, 0.1423417..."
2,3,CH,73431,A,17.00000,0,NaN,1.58426,1.26006,0.19016,...,da exigência de qualificação do trabalhador r...,da diminuição da importância da agricultura.,dos processos de desvalorização de áreas rurais.,NaN,NaN,NaN,maioria pessoa campo Vila Maria hoje exportado...,"[[-0.0246, -0.0379, 0.1021, 0.0378, -0.0864, -...","[0.08937166525923214, 0.12822456358498585, 0.1...","[[0.2161539, 0.16423477, -0.022779496, 0.32457..."
3,4,CH,58249,B,26.00000,0,NaN,1.87124,-0.40493,0.17401,...,"provocam o fenômeno da chuva ácida, devido ao...",são instalados próximos ao centro das cidades...,são responsáveis pelo desaparecimento das nas...,NaN,NaN,NaN,lixão mau tipo disposição final resíduo sólido...,"[[-0.0016, -0.0689, 0.0547, -0.0424, 0.0693, -...","[0.07309805460729854, 0.07309805460729854, 0.0...","[[-0.06490169, 0.25573078, 0.082524896, 0.1681..."
4,5,CH,73854,D,27.00000,0,NaN,0.40830,1.46209,0.02447,...,Implantação de curvas de nível.,"Aração do solo, do topo ao vale.",Terraceamento na propriedade,NaN,NaN,NaN,esquema representar processo erosão encosta pr...,"[[-0.0128, -0.0053, 0.029, -0.052, -0.0304, -0...","[0.238915125308662, 0.2103826054875806, 0.2674...","[[-0.123734914, 0.036533087, 0.20261528, 0.195..."


In [247]:
# salvar o objeto dataframe em um arquivo .pkl
file = os.path.join('data', 'df_itens_geral_validos.pkl')
with open(file, 'wb') as f:
    pickle.dump(data, f)

In [248]:
data.isnull().sum()

co_posicao                  0
sg_area                     0
co_item                     0
tx_gabarito                 0
co_habilidade              12
in_item_aban                0
tx_motivo_aban           1922
nu_param_a                  0
nu_param_b                  0
nu_param_c                  0
tx_cor                      0
co_prova                    0
id                          0
ano                         0
cor                         0
meio_aplicacao              0
num_questao                 0
area                      405
texto_questao               0
texto_base                266
alternativa_a             308
alternativa_b             114
alternativa_c             116
alternativa_d             112
alternativa_e              97
tp_lingua                1920
in_item_adaptado          722
tp_versao_digital        1517
texto_questao_tratado       0
wes                         0
tf_idf                      0
wes_own                     0
dtype: int64